<a href="https://colab.research.google.com/github/Georgeman31/std_projects/blob/main/RecSysForPharmacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install surprise pymorphy2 stop_words annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.0/648.0 KB 42.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 60.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 77.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=30640fb69dfbe8a35b117f2810fa10bcccae72d6555aa34879fc56ac1c3de3da
  Stored in directory: /root/.cache/pip/wheels/eb/03/0d/3bd31c983789aeb0b4d5e2ca48590288d9db1586cf5f225062
  Created wheel for annoy: filename=annoy-1.17.1-cp38-cp38-linux_x86_64.whl size=396893 sha256=5c243a2dfd88954c6c98a0edc25c6e2c70818509d375363a3e9f

In [3]:
# импорт библиотек
import pandas as pd
import io
import os
from google.colab import drive, files
import zipfile
import string
import annoy
import numpy as np
import pandas as pd
import surprise

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import FastText
from tqdm.auto import tqdm

# Загрузка и оценка данных

Таблица с чеками

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
zip_file = '/content/drive/MyDrive/чековые данные.zip'
z = zipfile.ZipFile(zip_file, 'r')
z.extractall()

print(os.listdir())

['.config', 'drive', 'τÑ¬«óδÑ ñá¡¡δÑ.csv', 'sample_data']


In [6]:
# переименование файла
os.rename('τÑ¬«óδÑ ñá¡¡δÑ.csv', 'checks.csv')

Описание продуктов

In [7]:
files.upload()

{}

Просмотр датасетов

In [7]:
df_products = pd.read_csv('products.csv')
df_products

,product_id,product
0,168308,(197312) Пакет-майка 25см х 45см 906
1,134832,(62448) Перекись водорода р-р наружн. 3% фл.по...
2,101384,(72183) Салициловая кислота р-р спирт 2% фл 40...
3,168570,(197309) Пакет 28см х 50см 906
4,146960,"(111023) Пакет ""Аптека Озерки"" 28 х 35см 906"
...,...,...
30413,193603,(110044169) название -1
30414,193484,(110050486) название -1
30415,192459,(110020427) название -1
30416,193510,(110047818) название -1


In [7]:
chunksize = 10 ** 6
tfr_checks=  pd.read_csv('checks.csv', chunksize=chunksize)

In [8]:
df_checks = pd.concat(tfr_checks, ignore_index=True)
df_checks

,sale_date_date,contact_id,shop_id,product_id,name,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455.0,168308.0,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00"
1,2018-12-07,198287,279.0,134832.0,(62448) Перекись водорода р-р наружн. 3% фл.по...,404.0,93.0,-1.0,"1,00"
2,2018-12-07,2418385,848.0,101384.0,(72183) Салициловая кислота р-р спирт 2% фл 40...,404.0,93.0,-1.0,"1,00"
3,2018-12-07,1285774,1511.0,168570.0,(197309) Пакет 28см х 50см,906.0,205.0,-1.0,"1,00"
4,2018-12-07,1810323,1501.0,168319.0,(197310) Пакет 30см х 60см,906.0,205.0,-1.0,"1,00"
...,...,...,...,...,...,...,...,...,...
19999998,2018-06-13,1924036,1485.0,71723.0,(60907) Тералиджен табл. п.п.о. 5 мг №25,637.0,146.0,-1.0,"1,00"
19999999,2018-06-13,1788779,1503.0,66164.0,(27922) Дифлюкан капс. 150 мг. №1,622.0,142.0,-1.0,"1,00"
20000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20000001,(затронуто стр,к: 20000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_checks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000003 entries, 0 to 20000002
Data columns (total 9 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   sale_date_date           object 
 1   contact_id               object 
 2   shop_id                  float64
 3   product_id               float64
 4   name                     object 
 5   product_sub_category_id  float64
 6   product_category_id      float64
 7   brand_id                 float64
 8   quantity                 object 
dtypes: float64(5), object(4)
memory usage: 1.3+ GB


# EDA

## Checks

Преобразование типов и избавление от пустых строк

In [10]:
df_checks.dropna(inplace=True)

In [11]:
df_checks = df_checks.astype({'contact_id': 'int32',
                        'shop_id': 'int16',
                        'product_id': 'int32',
                        'product_sub_category_id': 'int16',
                        'product_category_id': 'int16',
                        'brand_id': 'int16'}) 

In [12]:
df_checks.sale_date_date = pd.to_datetime(df_checks.sale_date_date)

In [13]:
df_checks['quantity'] = df_checks['quantity'].str.replace(',', '.').astype(float)

In [14]:
df_checks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19989262 entries, 0 to 19999999
Data columns (total 9 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   sale_date_date           datetime64[ns]
 1   contact_id               int32         
 2   shop_id                  int16         
 3   product_id               int32         
 4   name                     object        
 5   product_sub_category_id  int16         
 6   product_category_id      int16         
 7   brand_id                 int16         
 8   quantity                 float64       
dtypes: datetime64[ns](1), float64(1), int16(4), int32(2), object(1)
memory usage: 915.0+ MB


In [15]:
df_checks

,sale_date_date,contact_id,shop_id,product_id,name,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455,168308,(197312) Пакет-майка 25см х 45см,906,205,-1,1.0
1,2018-12-07,198287,279,134832,(62448) Перекись водорода р-р наружн. 3% фл.по...,404,93,-1,1.0
2,2018-12-07,2418385,848,101384,(72183) Салициловая кислота р-р спирт 2% фл 40...,404,93,-1,1.0
3,2018-12-07,1285774,1511,168570,(197309) Пакет 28см х 50см,906,205,-1,1.0
4,2018-12-07,1810323,1501,168319,(197310) Пакет 30см х 60см,906,205,-1,1.0
...,...,...,...,...,...,...,...,...,...
19999995,2018-06-13,1601618,1499,66842,(111992) Кэа Хэлс Ромашка [цветки фильтр-пакет...,615,140,1838,1.0
19999996,2018-06-13,1394104,1495,136795,(97857) Лориста Н таб. п.о. 50мг+12.5мг №90,738,170,-1,1.0
19999997,2018-06-13,1570654,1516,119513,(25299) Локрен тб п/о 20мг N28,738,170,-1,1.0
19999998,2018-06-13,1924036,1485,71723,(60907) Тералиджен табл. п.п.о. 5 мг №25,637,146,-1,1.0


## Products

In [18]:
df_NA = df_products[df_products['product'].str.contains('N/A -1')]
df_name = df_products[df_products['product'].str.contains('название -1')]

df_products_clean = df_products.drop(df_NA.index) #удалили все строки, содержащие N/A
df_products_clean = df_products_clean.drop(df_name.index) ##удалили все строки, содержащие "название -1"
df_products_clean.reset_index(drop=True, inplace=True) #замена индексации на сквозную
df_products_clean

,product_id,product
0,168308,(197312) Пакет-майка 25см х 45см 906
1,134832,(62448) Перекись водорода р-р наружн. 3% фл.по...
2,101384,(72183) Салициловая кислота р-р спирт 2% фл 40...
3,168570,(197309) Пакет 28см х 50см 906
4,146960,"(111023) Пакет ""Аптека Озерки"" 28 х 35см 906"
...,...,...
29595,153665,(121891) Очки корригирующие Кемнер Оптикс глян...
29596,164193,(190054) Лактулоза форте пор.15мл №10 597
29597,153013,(121465) Курапрокс Набор (зубная паста Вайт Из...
29598,59719,(117974) Лиерак Премиум Набор Бархат Новогодни...


In [19]:
#Убираем артикул продукта из названия
def remove_id(name):
  return name[name.find(')')+2:]

In [20]:
df_products_clean['product_name'] = [remove_id(x) for x in df_products_clean['product']]
df_products_clean

,product_id,product,product_name
0,168308,(197312) Пакет-майка 25см х 45см 906,Пакет-майка 25см х 45см 906
1,134832,(62448) Перекись водорода р-р наружн. 3% фл.по...,Перекись водорода р-р наружн. 3% фл.полимерн. ...
2,101384,(72183) Салициловая кислота р-р спирт 2% фл 40...,Салициловая кислота р-р спирт 2% фл 40мл N1 404
3,168570,(197309) Пакет 28см х 50см 906,Пакет 28см х 50см 906
4,146960,"(111023) Пакет ""Аптека Озерки"" 28 х 35см 906","Пакет ""Аптека Озерки"" 28 х 35см 906"
...,...,...,...
29595,153665,(121891) Очки корригирующие Кемнер Оптикс глян...,Очки корригирующие Кемнер Оптикс глянцевые пла...
29596,164193,(190054) Лактулоза форте пор.15мл №10 597,Лактулоза форте пор.15мл №10 597
29597,153013,(121465) Курапрокс Набор (зубная паста Вайт Из...,Курапрокс Набор (зубная паста Вайт Из Блэк 90м...
29598,59719,(117974) Лиерак Премиум Набор Бархат Новогодни...,Лиерак Премиум Набор Бархат Новогодний (крем 5...


In [17]:
df_checks.describe()

,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity
count,1.998926e+07,1.998926e+07,1.998926e+07,1.998926e+07,1.998926e+07,1.998926e+07,1.998926e+07
mean,1.022315e+06,6.684859e+02,9.983092e+04,5.821911e+02,1.334342e+02,4.072123e+02,1.397062e+00
std,7.015191e+05,5.056646e+02,4.464525e+04,1.636250e+02,3.761278e+01,9.930664e+02,3.971178e+00
min,3.730000e+02,1.000000e+00,8.086000e+03,-1.000000e+00,-1.000000e+00,-1.000000e+00,1.000000e-03
25%,4.582630e+05,3.070000e+02,6.504100e+04,4.580000e+02,1.080000e+02,-1.000000e+00,1.000000e+00
50%,9.216785e+05,3.500000e+02,1.000830e+05,6.130000e+02,1.400000e+02,-1.000000e+00,1.000000e+00
75%,1.418341e+06,9.460000e+02,1.343940e+05,7.030000e+02,1.620000e+02,-1.000000e+00,1.000000e+00
max,2.746262e+06,2.168000e+03,3.635860e+05,9.390000e+02,2.080000e+02,5.475000e+03,1.000000e+04


In [22]:
print('Unique product names:', df_products_clean['product_name'].nunique(), '\n')
print('Product statistics:', '\n', df_products_clean['product_name'].value_counts())

Unique product names: 27493 

Product statistics: 
 Лидокаина гидрохлорид р-р д/и 2% 2мл N10 399                          11
Салициловая кислота р-р спирт 2% фл 40мл N1 404                       10
Пиона настойка жидк фл 25мл N1 613                                    10
Винпоцетин таб.5мг №50 639                                            10
Бинт марл мед стер 5м х 10см уп N1 671                                10
                                                                      ..
Ополаскиватель д/полости рта Сплат Ультракомплекс 275мл 537            1
Нюкс Рэв де Мьель Стик д/губ увлажняющий 4г 500                        1
Цераве Вода мицеллярная 296мл 500                                      1
Хна Иранская натуральная 25г 479                                       1
Бельведер Тушь д/ресниц антибактериальная кремовая черная 10мл 498     1
Name: product_name, Length: 27493, dtype: int64


# Обучение

## **Embbedings**

In [41]:
# Для фильтрации пунктуации
exclude = set(string.punctuation)
# Для приведения слов в начальной форме
morpher = MorphAnalyzer()

# Для фильтрации стоп-слов
sw = get_stop_words("ru")

def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [24]:
# Загрузим текстовые описания товаров и преобразуем к нормализованному виду
df_products_clean['text'] = df_products_clean['product_name'].apply(lambda x: preprocess_txt(str(x)))

# Обучим модель fasttext на текстах
sentences = [i for i in df_products_clean['text'] if len(i) > 2]
modelFT = FastText(sentences=sentences, size=20, min_count=1, window=5)

In [27]:
def recommend(item_id):
    # Получим тайтл айтема по идентификатору
    title = df_products_clean[df_checks['product_id']==item_id]['product_name'].values[0]
    title = preprocess_txt(title)

    i = df_products_clean[df_products_clean['product_id']==item_id].index[0]
    
    vector_ft = np.zeros(20)
    n_ft = 0
    # Каждое слово обернем в эмбеддинг
    for word in df_products_clean.loc[i, "text"]:
        if word in modelFT:
            vector_ft += modelFT[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    # Найдем ближайшие по тайтлу
    rec_items = ft_index_shop.get_nns_by_vector(vector_ft, 5)
    return [index_map_shop[i][0] for i in rec_items]

In [26]:
ft_index_shop = annoy.AnnoyIndex(20 ,'angular')

index_map_shop = {}
counter = 0

for i in tqdm(range(len(df_products_clean))):
    n_ft = 0
    index_map_shop[counter] = (df_products_clean.loc[i, "product"], df_products_clean.loc[i, "product_id"])
    vector_ft = np.zeros(20)
    # Каждое слово обернем в эмбеддинг
    for word in df_products_clean.loc[i, "text"]:
        if word in modelFT:
            vector_ft += modelFT[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1

# 
ft_index_shop.build(10)

  0%|          | 0/29600 [00:00<?, ?it/s]

<ipython-input-26-8587d8e002fd>:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if word in modelFT:
<ipython-input-26-8587d8e002fd>:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector_ft += modelFT[word]


True

Рекоммендация с помощью модели

In [31]:
item_id = 146960

print(df_products_clean[df_products_clean['product_id']==item_id]['product'], '\n')

recommend(item_id)

4    (111023) Пакет "Аптека Озерки" 28 х 35см 906
Name: product, dtype: object 



<ipython-input-27-b0d5e052741a>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  title = df_products_clean[df_checks['product_id']==item_id]['product_name'].values[0]
<ipython-input-27-b0d5e052741a>:12: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if word in modelFT:
<ipython-input-27-b0d5e052741a>:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector_ft += modelFT[word]


['(111023) Пакет "Аптека Озерки" 28 х 35см 906',
 '(111024) Пакет "Аптека Озерки" 38 х 50см 906',
 '(259713) Пакет "Аптека Озерки" 24 х 45см ПНД 917',
 '(205414) Пакет "Озерки Дрогери" 24 х 30см ПНД 906',
 '(205037) Пакет Радуга Кидс п/э 29 х 40см 906']

## **SVD**

Попробуем воспользоваться моделью SVD для построения рекомендации

In [133]:
# возьмем первые 10.000 уникальных значений клиентов для улучшения производительности
df_split = df_checks.drop_duplicates('contact_id')[:10000]

In [134]:
# определим переменные для оценки рекоммендации
df_learn_checks = df_split.rename(columns={'contact_id': 'uid', 'product_id': 'iid', 'quantity': 'rating'})[['uid', 'iid', 'rating']]

ui_reader = surprise.Reader(rating_scale=(0, 1.0))
ds_svd_predict = surprise.Dataset.load_from_df(df_learn_checks, ui_reader)

In [135]:
# разбиение выборки
train, test = surprise.model_selection.train_test_split(ds_svd_predict, test_size=0.3, shuffle=False)

In [136]:
# обучение модели
recs_svd_model = surprise.SVD()
recs_svd_model.fit(train)

In [137]:
# прогон тестовых данных
pred_svd = recs_svd_model.test(test)

In [138]:
# получим оценку корня среднеквадратичного отклонения
surprise.accuracy.rmse(pred_svd, verbose=True)

RMSE: 1.5929


1.592913422501849

Получили относительно низкое отклонение

In [141]:
# получим предсказание
recs_svd_model.predict(iid=136795, uid=2418385, verbose=True)

user: 2418385    item: 136795     r_ui = None   est = 1.00   {'was_impossible': False}


Prediction(uid=2418385, iid=136795, r_ui=None, est=1.0, details={'was_impossible': False})

# Метрики качества

In [142]:
def precision_5(recommended, bought):
  '''precision@5: количество купленных товаров из рекоммендованного / 5'''
  qnt = 0

  for i in recommended:
    if i in bought:
      qnt += 1
  
  return qnt/5

def recall(recommended, bought):
  '''recall: количество купленных товаров из рекоммендованного / количество купленных товаров'''
  qnt = 0

  for i in recommended:
    if i in bought:
      qnt += 1
  
  return qnt/len(bought)

In [149]:
bought = [134832, 101384, 168570, 146960, 164193, 54876]
recommended = [134832, 101384, 146960, 165489]

print('Precision: ' + str(precision_5(recommended, bought)) + ' Recall: ' + str(recall(recommended, bought)))

Precision: 0.6 Recall: 0.5
